In [1]:
import beta_rec

In [2]:
from beta_rec.utils.evaluation import *

In [3]:
import pandas as pd
import numpy as np

In [4]:
df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

In [5]:
df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

In [6]:
df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

In [7]:
df_pred_new.fillna(0, inplace=True)

In [8]:
df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [9]:
df_pred_new

,user_id,item_id,score,relevance
40,3,590,0.280305,0.0
41,3,1136,0.263707,0.0
42,3,920,0.231234,0.0
43,3,1246,0.188023,0.0
44,3,318,0.187111,0.0
...,...,...,...,...
1179555,71534,1304,0.088851,0.0
1179556,71534,778,0.085875,0.0
1179557,71534,1307,0.082908,0.0
1179558,71534,1258,0.082717,0.0


In [21]:
df_test

,user_id,item_id,rating,timestamp,relevance
0,1256,5378,5.0,2007-10-31 12:18:24,5.0
1,1256,778,4.5,2007-10-31 12:19:51,4.5
2,1256,8376,5.0,2007-10-31 12:30:59,5.0
3,1256,2594,5.0,2007-10-31 12:20:49,5.0
4,1256,30810,4.5,2007-10-31 12:32:24,4.5
...,...,...,...,...,...
38806,68571,112,5.0,2007-01-28 01:37:59,5.0
38807,68571,7256,5.0,2007-01-28 01:36:16,5.0
38808,68571,34405,4.5,2007-01-28 01:13:53,4.5
38809,68571,2618,5.0,2007-01-28 01:43:19,5.0


In [24]:
precision_at_k(
    df_test,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.0575192096597146

In [25]:
recall_at_k(
    df_test,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.09632131280310642

In [26]:
map_at_k(
    df_test,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.031690245847362415

In [27]:
ndcg_at_k(
    df_test,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.09326437896736813

In [33]:
df_test['relevance'] = df_test.relevance.astype(int)

In [39]:
auc(
    df_pred_new,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
)

0.5305776379264664